In [ ]:
!pip install langchain
!pip install tiktoken
!pip install -U langchain langchain-community
!pip install -U langchain langchain-core langchain-openai
!pip install chromadb
!pip install -U langchain sentence-transformers chromadb
!pip install -U sentence-transformers langchain

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_cublas_

In [ ]:
# model
llm = ChatOpenAI(model="gpt-4o-mini")

# chain 실행
response = llm.invoke("지구의 자전 주기는? 50글자 아내로 말해줘")

In [ ]:
response.content

'지구의 자전 주기는 약 24시간입니다. 정확히 말하면, 태양을 기준으로 할 때의 자전 주기는 약 24시간(1일)이며, 이를 **태양일**이라고 합니다. 반면, 별을 기준으로 하는 자전 주기는 약 23시간 56분 4초로, 이를 **항성일**이라고 합니다. 태양일과 항성일의 차이는 지구가 자전하는 동안 태양이 우리 지구를 기준으로 이동하기 때문입니다.'

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
# model
prompt = ChatPromptTemplate.from_template("너는 천문에 전문가야. 질문에 50자 이내로 답변해줘. <Question>: {input}")
llm = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input" : "지구의 자전 주기는?"})

'지구의 자전 주기는 약 24시간입니다.'

In [ ]:
prompt_one = ChatPromptTemplate.from_template("{한글 단어}을 영어로 번역만 해줘")
prompt_two = ChatPromptTemplate.from_template("{영어}를 옥스퍼드 사전을 사용해서 나에게 한글로 설명해줘")

llm = ChatOpenAI(model="gpt-4o-mini")

chain_one = prompt_one | llm | StrOutputParser()


chain_one.invoke({"한글 단어":"미래"})

'Future'

In [ ]:
chain_two = (
    {"영어" : chain_one}
    | prompt_two
    | llm
    | StrOutputParser()
)

chain_two.invoke({"한글 단어":"미래"})

'"Future"는 옥스퍼드 사전에 따르면 "앞으로 일어날 것, 즉 미지의 시간이거나 아직 발생하지 않은 사건이나 상황"을 의미합니다. 또한 "미래"라는 개념은 시간의 흐름 속에서 현재를 지나며 다가오는 시점이나 특정한 시기를 말합니다. "Future"는 또한 개인이나 사회가 어떤 방향으로 나아갈지를 나타내는 가능성과 기회를 포함하기도 합니다.'

In [ ]:
prompt_three = ChatPromptTemplate.from_template("지구과학에서 {topic}에 대해 간단히 설명해줘. 50글자 이내로")
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

chain_three = prompt_three|model|output_parser

topics = ["지구 공전", "화산 활동", "대륙 이동"]
results = chain_three.batch([{"topic": t} for t in topics])
for topic, result in zip(topics, results):
    print(f"Topic: {topic}\nResult: {result}\n")

Topic: 지구 공전
Result: 지구 공전은 지구가 태양 주위를 한 바퀴 도는 운동으로, 한 해에 약 365.25일 걸립니다.

Topic: 화산 활동
Result: 화산 활동은 마그마가 지표로 분출되는 현상으로, 화산 eruptions, 용암 흐름, 화산 구조 형성을 포함한다.

Topic: 대륙 이동
Result: 대륙 이동설은 대륙이 지구의 표면에서 천천히 이동한다는 이론으로, 판 tectonics에 기반한다.



In [ ]:
stream = chain_three.stream({"topic": "지진"})
print("stream 결과: ")
for chunk in stream:
    print(chunk, end="", flush=True)
print()

stream 결과: 
지진은 지구의 땅속에서 발생하는 에너지가 방출되며, 땅이 흔들리는 현상입니다. 주로 판구조론과 관련이 있습니다.


In [ ]:
import nest_asyncio
import asyncio

In [ ]:
nest_asyncio.apply()

In [ ]:
async def run_async():
  result = await chain_three.ainvoke({"topic": "지진"})
  print("ainvoke 결과:", result[:50], "...")

asyncio.run(run_async())

ainvoke 결과: 지진은 Earth's crust에서의 갑작스러운 에너지 방출로 발생하며, 파동이 전파되어  ...


In [ ]:
from  langchain_core.prompts import PromptTemplate

template_text = "안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다."

prompt_template = PromptTemplate.from_template(template_text)

filled_prompt = prompt_template.format(name="홍길동", age=30)

In [ ]:
filled_prompt

'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.'

In [ ]:
combined_prompt = (
    prompt_template
      + PromptTemplate.from_template("\n\n아버지를 아버지라 부를 수 없습니다.")
        + "\n\n{language}로 번역해주세요."
)

In [ ]:
combined_prompt

PromptTemplate(input_variables=['age', 'language', 'name'], input_types={}, partial_variables={}, template='안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n{language}로 번역해주세요.')

In [ ]:
combined_prompt.format(name="홍길동", age=30, language="영어")

'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n영어로 번역해주세요.'

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini")
chain = combined_prompt | llm | StrOutputParser()
chain.invoke({"name":"홍길동", "language":"영어", "age":"30"})

'Hello, my name is Hong Gil-dong, and I am 30 years old.\n\nI cannot call my father "father."'

###데이터 로드
- url을 통해서 웹사이트 정보 가져오기

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'
loader = WebBaseLoader(url)

docs = loader.load()


### 분할 (text split)
- url에서 불러온 텍스트를 분할하여 검색하기 쉬운 구조로 분할

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

print(len(splits))
print(splits[10])

19
page_content='제안과 채택
 백:아님 § 관료주의  문서를 참고하십시오. 단축백:제안
제안 문서란 정책과 지침으로 채택하자고 의견을 묻는 문서이나 아직 위키백과 내에 받아들여지는 원칙으로 확립되지는 않은 문서입니다. {{제안}} 틀을 붙여 공동체 내에서 정책이나 지침으로 채택할 지 의견을 물을 수 있습니다. 제안 문서는 정책과 지침이 아니므로 아무리 실제 있는 정책이나 지침을 요약하거나 인용해서 다른 문서에 쓴다고 해도 함부로 정책이나 지침 틀을 붙여서는 안 됩니다.
'제안'은 완전 새로운 원칙이라기보다, 기존의 불문율이나 토론 총의의 문서를 통한 구체화에 가깝습니다. 많은 사람들이 쉽게 제안을 받아들이도록 하기 위해서는, 기초적인 원칙을 우선 정하고 기본 틀을 짜야 합니다. 정책과 지침의 기본 원칙은 "왜 지켜야 하는가?", "어떻게 지켜야 하는가?" 두 가지입니다. 특정 원칙을 정책이나 지침으로 확립하기 위해서는 우선 저 두 가지 물음에 성실하게 답하는 제안 문서를 작성해야 합니다.
좋은 아이디어를 싣기 위해 사랑방이나 관련 위키프로젝트에 도움을 구해 피드백을 요청할 수 있습니다. 이 과정에서 공동체가 어느 정도 받아들일 수 있는 원칙이 구체화됩니다. 많은 이와의 토론을 통해 공감대가 형성되고 제안을 개선할 수 있습니다.
정책이나 지침은 위키백과 내의 모든 편집자들에게 적용되는 원칙이므로 높은 수준의 총의가 요구됩니다. 제안 문서가 잘 짜여졌고 충분히 논의되었다면, 더 많은 공동체의 편집자와 논의를 하기 위해 승격 제안을 올려야 합니다. 제안 문서 맨 위에 {{제안}}을 붙여 제안 안건임을 알려주고, 토론 문서에 {{의견 요청}}을 붙인 뒤 채택 제안에 관한 토론 문단을 새로 만들면 됩니다. 많은 편집자들에게 알리기 위해 관련 내용을 {{위키백과 소식}}에 올리고 사랑방에 이를 공지해야 하며, 합의가 있을 경우 미디어위키의 sitenotice(위키백과 최상단에 노출되는 구역)에 공지할 수도 있습니다.' metadata={'source': '

### 인덱싱(Indexing)
- openAI에서 제공하는 인베딩 AI 모델은 비용 발생 => 무료 제공되는 오픈된 AI 모델 사용
- 임베딩 모델 설정
- vectorstore를 활용해서 유사한 문장 인덱싱

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embedding_model = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS"
)

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding_model,
    persist_directory="./chroma_kr_sbert"  # 새 디렉토리
)

query = "격하 과정에 대해서 설명해주세요."
docs = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(docs):
    print(f"[{i+1}] 유사 문서 내용:\n{doc.page_content}\n")


/tmp/ipython-input-6-1997673724.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[1] 유사 문서 내용:
차후 공지가 불충분했다는 이의 제기를 피하려면, 위의 링크를 이용하여 공지하세요. 공지에 비중립적인 단어를 사용하는 등의 선전 행위는 피하세요.
토론이 끝났다면 선언과 함께 {{토론보존}} 틀을 이용하여 닫습니다. 총의 판단은 여타 토론과 마찬가지로  분쟁 해결 정책에서 갈음해 처리합니다. 토론을 통해 정책이나 지침 채택 여부를 논의하며, 이 과정에서 제안 문서가 크게 수정될 수도 있습니다. 토론 중 제안을 정식 정책/지침으로 채택하자는 합의로 모이고 나서 2주 (정확히 14일. 이후 내용은 모두 같습니다) 간 제안을 대폭 수정해야 하는 변경안 제시나 명확한 근거가 존재하는 반대가 나오지 않는다면 정책이나 지침으로 정식으로 채택됩니다. 반대로 토론자들 사이에서 채택을 거부한다는 합의가 모아져서 2주간 명확한 근거가 존재하는 반대 의견이 나오지 않는다면 채택안 거부 총의가 모아졌다고 보아 기각됩니다. 주요한 총의 판단 기준은 다음과 같습니다.

[2] 유사 문서 내용:
실질적인 변경
실행하세요. 정책 및 지침 문서를 실질적으로 변경하기 전에, 기존 관행에 대한 합리적인 예외를 설정하는 것이 유용할 수 있습니다. 이러한 방식으로 기존의 관행을 갱신하기 위해, 백:과감과 백:무시 정신에 따라 기존의 관행에서 직접적으로 벗어날 수도 있습니다. 시간이 지나고, 변경에 대한 이의가 없거나 토론을 통해 변경 또는 구현에 대한 광범위한 총의에 도달한 경우, 관행을 설명하는 정책 및 지침 문서를 편집하여 새로운 상황을 반영할 수 있습니다.
토론 먼저 하세요. 토론 문서 내 논의는 일반적으로 정책의 실질적인 변경보다 우선합니다. 이의가 없거나 토론에서 해당 변경에 대한 총의가 형성되었다면 변경할 수 있습니다. 서식, 문법 및 명료성 개선을 위한 사소한 편집은 언제든지 가능합니다.
논의 결과가 불분명할 경우 제안 과정에서처럼 관리자나 다른 독립 편집자가 평가해야 합니다. 주요한 변화들은 또한 일반적으로 지역사회에 공표되어야 합니다; 제안 과정과 유사한 발

### 생성(generation)
- prompt 생성
- 모델 설정
- 유사 문장 검색
- 검색 문장 합
- rag_chain 적용
- 사용

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# LLM
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# Rretriever
retriever = vectorstore.as_retriever()

# Combine Documents
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
rag_chain.invoke("격하 과정에 대해서 설명해주세요.")


'격하 과정은 특정 정책이나 지침이 편집 관행이나 공동체 규범의 변화로 인해 쓸모없어지거나, 다른 문서와 내용이 중복될 때 진행됩니다. 이 과정은 다음과 같은 단계로 이루어집니다:\n\n1. **논의 시작**: 편집자들은 정책을 지침으로 격하하거나 보충 설명, 정보문, 수필 또는 중단 문서로 격하할 것을 제안합니다. 이를 위해 토론 문서 내에서 논의를 시작하고, 프로젝트 문서 상단에 {{새로운 토론|문단=진행 중인 토론 문단}} 틀을 붙여 공동체의 참여를 요청합니다.\n\n2. **논의 진행**: 충분한 논의가 이루어진 후, 제3의 편집자가 토론을 종료하고 평가하여 상태 변경에 대한 총의가 형성되었는지 판단합니다.\n\n3. **상태 변경**: 폐지된 정책이나 지침은 최상단에 {{중단}} 틀을 붙여 더 이상 사용하지 않는 정책/지침임을 알립니다.\n\n이러한 과정을 통해 정책이나 지침의 유효성을 재검토하고, 필요에 따라 적절한 조치를 취할 수 있습니다.'

### 랭체인의 기본 작동 방법

In [ ]:
## 합본

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import WebBaseLoader

#url 바탕으로 사이트의 정보를 가져온다.
url = 'https://wikidocs.net/231429'
loader = WebBaseLoader(url)

# 가져온 사이트 정보에서 텍스트를 추출하고 분할하여 학습 및 검색을 하기 손쉽게 만들어 준다. chunk_over을 통해서 문맥이 끊기지 않게 도와준다.
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 분할시킨 텍스트를 오픈된 AI모델을 활용해서 벡터로 값을 변환하여 임베딩한다. 이후에 인덱싱을 해서 검색하기 쉽게 변환한다.
embedding_model = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS"
)


vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding_model,
    persist_directory="./chroma_kr_sbert"  # 새 디렉토리
)

# Prompt 생성 {context : 검색할 내용}, {question : 사용자 입력 검색어}
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# LLM : 검색해둔 값을 바탕으로 정보를 판단할 LLM을 설정한다.
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# Rretriever
# 벡터값을 이용해서 질문과 가장 비슷한 문단을 찾는다.
retriever = vectorstore.as_retriever()

# Combine Documents
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
rag_chain.invoke("데이터 변환 및 정제에 대해서 설명해주세요.")

'데이터 변환 및 정제는 불러온 문서 데이터를 분석하고 처리하여, 랭체인의 다른 모듈이나 알고리즘이 처리하기 쉬운 형태로 변환하는 과정을 포함합니다. 이 과정에서는 불필요한 데이터를 제거하거나, 데이터의 구조를 변경할 수 있습니다. 이를 통해 데이터의 품질을 높이고, 이후의 분석이나 처리가 원활하게 이루어질 수 있도록 합니다.'